In [ ]:
import glob
import os

import cv2
from path import Path
from tqdm import tqdm

In [ ]:
os.chdir('..')

In [ ]:
FOLDER = Path('/media/vitaliy/Samsung_T5/y')
SAVE_FILE = FOLDER / 'img_dims.tsv'

In [ ]:
paths = glob.glob(os.path.join(FOLDER, '**', '*'), recursive=True)

In [ ]:
paths_from_file: set[str] = set()
if SAVE_FILE.exists():
    with open(SAVE_FILE, mode='rt', encoding='utf-8') as fp:
        for line in fp:
            paths_from_file.add(line.strip().split('\t')[0])

In [ ]:
cnt = 0
existing_short_paths = set()

with open(SAVE_FILE, 'at', encoding='utf-8') as fp:
    for path in tqdm(paths):
        short_path = FOLDER.relpathto(path)
        existing_short_paths.add(short_path)
        
        if short_path in paths_from_file:
            continue
        
        img_name, extension = os.path.splitext(os.path.basename(path))
    
        if extension.lower() not in ('.jpg', '.jpeg', '.png'):
            continue
    
        img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
        dims = img.shape
        fp.write(f'{short_path}\t{dims[0]}\t{dims[1]}\n')
        cnt += 1
        
print(f'processed: {cnt} files')

# Get statistics

In [ ]:
grouped_by_height = {}

with open(SAVE_FILE, mode='rt', encoding='utf-8') as fp:
    for line in fp:
        image_short_path, height, _ = line.strip().split('\t')
        
        if image_short_path not in existing_short_paths:
            continue
        
        height = int(height)
        folder = str(Path(image_short_path).parent)
        
        if height not in grouped_by_height:
            grouped_by_height[height] = {'folders': set(), 'files': set()}
        
        grouped_by_height[height]['folders'].add(folder)
        grouped_by_height[height]['files'].add(image_short_path)

In [ ]:
for height in sorted(grouped_by_height.keys(), reverse=False)[:5]:
    folders = sorted(grouped_by_height[height]['folders'])
    files = [str(FOLDER / file) for file in sorted(grouped_by_height[height]['files'])]
    print(f'height: {height}\nfolders: {folders}\nfiles: {files}\n\n')

In [ ]:
!python inference_realesrgan.py -n RealESRGAN_x4plus -i "/media/vitaliy/Samsung_T5/" \
--outscale 4 --min_height 400 --suffix upscaled --face_enhance --save_logs